<a id='toc'></a>

# Table of Contents
[Imports](#imports)<br>
[Load Data](#load_data)<br>
[Get Building Locations](#bldg_locations)<br>
[Get Data from GitHub](#github_data)<br>
[Load and Query AWS Database](#aws_db_cxn)<br>
[Join Generalised Building Names with Original Org Table](#join_gen_name)<br>
[Get Distances between Patients and Medical Facilities](#dist_calcs)<br>
[Get Distance Statistics and Closest Medical Organisation for Each Patient](#dist_stats)<br>
[Map Visualisation for Patient Locations](#map_viz)<br>
[Box Plots for a Selected Distance Range](#boxplots_gt10km)<br>
[EDA for Modelling - Automated Profile Reports](#eda_modelling_auto)<br>
[EDA for Modelling - Simple Pandas Dataframes](#eda_modelling_simple)<br>
[Primary Keys](#pri_keys)<br>
[Create ML Tables](#create_ML_tables)<br>
[Create and Join Pivot Tables for All Tables on PATIENT](#pivot_patient)<br>
[Random Forest Classifier Data Preparation](#rfc_prep)<br>
[Random Forest Classifier Modelling](#rfc_model)<br>
[Quasi ERD](#quasi_ERD)<br>



<a id='imports'></a>

# Imports
[Back to TOC](#toc)<br>

In [ ]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
import datetime
import requests
import plotly.express as px
import pymysql; pymysql.install_as_MySQLdb()
from io import BytesIO
from sqlalchemy import create_engine
from time import time
import pdb
import pandas_profiling
from ipywidgets import IntProgress
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score, roc_curve

# Measure run time for whole notebook.
start_time = time()


<a id='load_data'></a>

# Load Data
[Back to TOC](#toc)<br>

In [ ]:
# Load all data.
allergies = pd.read_csv('allergies.csv')
careplans = pd.read_csv('careplans.csv')
conditions = pd.read_csv('conditions.csv')
devices = pd.read_csv('devices.csv')
encounters = pd.read_csv('encounters.csv')
imaging_studies = pd.read_csv('imaging_studies.csv')
immunizations = pd.read_csv('immunizations.csv')
medications = pd.read_csv('medications.csv')
observations = pd.read_csv('observations.csv')
organizations = pd.read_csv('organizations.csv')
patients = pd.read_csv('patients.csv')
payers = pd.read_csv('payers.csv')
payer_transitions = pd.read_csv('payer_transitions.csv')
procedures = pd.read_csv('procedures.csv')
providers = pd.read_csv('providers.csv')
supplies = pd.read_csv('supplies.csv')


<a id='bldg_locations'></a>

# Get Building Locations
[Back to TOC](#toc)<br>

In [ ]:
# Purpose: Build a lookup table for geo-coordinates of all facilities (e.g. hospitals and medical centres).
orgs = pd.read_csv('organizations.csv')
display(orgs)

# Get unique facilities based on Lat and Long values.
grp_orgs_lat_long = (orgs.groupby(['LAT', 'LON']).
                     count()[['Id']].
                     rename(columns={'Id' : 'COUNT'}).
                     sort_values('COUNT', ascending=False)
                    )

# Make up generic building names, since there is not a 1:1 relationship between NAME and LAT-LONG pairs.
bldg_names = [f'Building_{str(i+1).zfill(3)}' for i in range(len(grp_orgs_lat_long))]
grp_orgs_lat_long['NAME_GENERALISED'] = bldg_names
org_lookup = grp_orgs_lat_long.reset_index()[['NAME_GENERALISED', 'LAT', 'LON', 'COUNT']]
display(org_lookup)

# Save for manual transfer to GitHub repo (already uploaded).
# https://github.com/signorec/LGC-Demo.git
org_lookup.to_csv('org_lookup.csv', index=False, header=True)


<a id='github_data'></a>

# Get Data from GitHub
[Back to TOC](#toc)<br>


In [ ]:
# Download specified file from GitHub, regardless of file type.
def download_github_file(readfile):
    # GitHub public demo repo.
    readurl = "https://raw.githubusercontent.com/signorec/LGC-Demo/main/{}?raw=true".format(readfile)

    # Stream file bytes.
    readfile_bytes = BytesIO(requests.get(readurl).content)
    try:
        # Write file to local machine.
        write_path = readfile
        with open(write_path, 'wb') as bin_file:
            bin_file.write(readfile_bytes.read())
            
    except Exception as e:
        print('Could not download file!')
        print(e)
        
    else:
        print('File saved: ' + readfile)
        
# Download CSV with medical building geocoordinates.
download_github_file('org_lookup.csv')


<a id='aws_db_cxn'></a>

# Load and Query AWS Database
[Back to TOC](#toc)<br>

In [ ]:
# AWS RDS MySQL DB Connection Details.
user = 'lgcsignorec'
password = 'lgcsignorec'
host = 'lgcsignorecdb.cx4gjl4pip7t.eu-west-1.rds.amazonaws.com'
port = '3306'
dbname = 'lgcsignorecdb'

# From MySQL Shell:
# \connect lgcsignorec:lgcsignorec@lgcsignorecdb.cx4gjl4pip7t.eu-west-1.rds.amazonaws.com:3306

# Connect to DB.
RDS_MySQL_engine = create_engine(f'mysql+mysqldb://{user}:{password}@{host}:{port}/{dbname}')

# Close DB connection implicitly using 'with..as'
with RDS_MySQL_engine.connect() as RDS_MySQL_cxn:
    # Load medical building lookup table into DB.
    data_to_insert = pd.read_csv('org_lookup.csv')
    print(f'Data loaded into AWS MySQL DB:\n{user}@{host}:{port}\n')
    print(data_to_insert)

    # Insert data
    SQL_table_name = 'ORG_LOOKUP'
    data_to_insert.to_sql(SQL_table_name, RDS_MySQL_cxn, if_exists='replace', index=False)

    # Query data: get top 9 most used facilities.
    simple_qry = \
    '''
    SELECT NAME_GENERALISED, LAT, LON
    FROM ORG_LOOKUP
    WHERE NAME_GENERALISED LIKE 'Building_00%%';
    '''
    qry_result = RDS_MySQL_cxn.execute(simple_qry).fetchall()

    print(f'\nData queried from AWS MySQL DB.\nLocations of Top 9 Most Used Medical Facilities:\n')
    for bldg_name, lat, lon in qry_result:
        print(f'County:\t{bldg_name:15}\tLatitude:{lat:>15}\tLongitude:{lon:>15}')


<a id='join_gen_name'></a>

# Join Generalised Building Names with Original Org Table
[Back to TOC](#toc)<br>

In [ ]:
org_lookup = pd.read_csv('org_lookup.csv')
orgs = pd.read_csv('organizations.csv')

orgs_with_gen_names = orgs.merge(org_lookup,
                                 how='inner',
                                 on=['LAT', 'LON'],
                                 indicator=True)

# Show the mapped building names.
display(orgs_with_gen_names[['LAT', 'LON', 'NAME', 'NAME_GENERALISED']].sort_values('NAME_GENERALISED'))

# Check that all data matched - should be only 'both' entries
print('Checking mapping relationship:')
display(orgs_with_gen_names['_merge'].value_counts())


<a id='dist_calcs'></a>

# Get Distances between Patients and Medical Facilities
[Back to TOC](#toc)<br>

In [ ]:
######################################################################
# Function: Lambert_dist_calc()
#
# Calculates the line-of-sight distance between two points defined by latitude and longitude.
# Assumes cols exist in df for Building LAT/LON and Patient LAT/LON
#
# Uses Lambert's formula.
# See also Bowring's method for short lines.
# https://en.wikipedia.org/wiki/Geographical_distance
#
# Input: row: pandas data frame row passed in by apply() function.
# Output: single distanace value in km between building and patient geocoordinates.
#
# Example USAGE: df['Distance'] = df.apply(Lambert_dist_calc, axis=1)
#
######################################################################
def Lambert_dist_calc(row):
    try:
        phi_1 = math.radians(row['Building LAT']) # latitude of point 1.
        phi_2 = math.radians(row['Patient LAT']) # latitude of point 2.
        d_phi = phi_1 - phi_2

        lambda_1 = math.radians(row['Building LON']) # longitude of point 1.
        lambda_2 = math.radians(row['Patient LON']) # longitude of point 2.
        d_lambda = lambda_1 - lambda_2

        # Central angle in radians between two points on a sphere using the Great-circle distance method.
        # https://en.wikipedia.org/wiki/Great-circle_distance
        sigma = math.acos(math.sin(phi_1)*math.sin(phi_2) + \
                          math.cos(phi_1)*math.cos(phi_2)*math.cos(d_lambda))

        # Flattening factor.
        f = 1/298.257223563

        # https://en.wikipedia.org/wiki/Latitude#Parametric_(or_reduced)_latitude
        beta_1 = math.atan((1 - f)*math.tan(phi_1))
        beta_2 = math.atan((1 - f)*math.tan(phi_2))

        P = (beta_1 + beta_2)/2
        Q = (beta_1 - beta_2)/2

        X = (sigma - math.sin(sigma)) * (math.sin(P))**2 * (math.cos(Q))**2 / (math.cos(sigma/2))**2
        Y = (sigma + math.sin(sigma)) * (math.cos(P))**2 * (math.sin(Q))**2 / (math.sin(sigma/2))**2

        # radius of the earth (km)
        a = 6371.009
        dist = a*(sigma - f/2*(X + Y))

    except:
        dist = np.nan

    return dist

# Load lookup table with distinct building locations of organisations.
org_lookup = pd.read_csv('org_lookup.csv')
building_locs = org_lookup[['NAME_GENERALISED', 'LAT', 'LON']]

# Load patient locations.
patients = pd.read_csv('patients.csv')
patient_locs = patients[['Id', 'LAT', 'LON']]

# Measure execution time.
start_time = time()

# Join tables to get all combinations of building-patient pairs.
all_combs = (pd.merge(building_locs, patient_locs, how='cross')
             .rename(columns={'NAME_GENERALISED' : 'Building ID',
             'LAT_x' : 'Building LAT',
             'LON_x' : 'Building LON',
             'Id' : 'Patient ID',
             'LAT_y' : 'Patient LAT',
             'LON_y' : 'Patient LON'
             })
            )

# Calculate distances between buildings and patients.
all_combs['Distance [km]'] = all_combs.apply(Lambert_dist_calc, axis=1)
all_combs.to_csv('patient_building_distances.csv', index=False, header=True)

# Join and calculation time is about 125 sec on personal laptop.
time_diff = time() - start_time
print(f'Execution Time [s]:\t{time_diff}')

display(all_combs.sort_values('Distance [km]'))


<a id='dist_stats'></a>

# Get Distance Statistics and Closest Medical Organisation for Each Patient
[Back to TOC](#toc)<br>

In [ ]:
# Show quartile statistics for distances between patients and buildings for all medical organisations.
# Percentile_0 = Minimum
# Percentile_25 = Lower end of the inter-quartile range (IQR).
# Percentile_50 = Median
# Percentile_75 = Upper end of the inter-quartile range (IQR).
# Percentile_100 = Maximum

def percentile(n):
    def percentile_(x):
        return np.percentile(x, n)
    percentile_.__name__ = 'Percentile_%s' % n
    return percentile_

# Load combined patient and organisation data with distances between them.
all_combs_in = pd.read_csv('patient_building_distances.csv')

print('Ordered Distances for Each Patient:')
display(all_combs_in.sort_values(['Patient ID', 'Distance [km]']))

# Group such that quartile statistics can be tabulated.
grp_pat_bldg_dist = (all_combs_in.groupby('Patient ID')['Distance [km]']
                     .agg([percentile(0),
                           percentile(25),
                           percentile(50),
                           percentile(75),
                           percentile(100)
                          ])
                    )

# Attach the building identifier of the closest organisation to each patient.
dist_stats_and_closest = (pd.merge(grp_pat_bldg_dist,
                                   all_combs_in,
                                   how='left',
                                   left_on='Percentile_0',
                                   right_on='Distance [km]')
                          .rename(columns={'Building ID' : 'Closest Med. Org.'})
                          [['Patient ID',
                            'Closest Med. Org.',
                            'Percentile_0',
                            'Percentile_25',
                            'Percentile_50',
                            'Percentile_75',
                            'Percentile_100']]
                         ).sort_values('Percentile_0')

print('Quartile Statistics for Distances [km] between Patients and Medical Organisations.')
print('Closest medical organisation also shown.')
display(dist_stats_and_closest)

# Save to file.
dist_stats_and_closest.to_csv('pat_bldg_dist_stats.csv', index=False, header=True)


<a id='map_viz'></a>

# Map Visualisation for Patient Locations
[Back to TOC](#toc)<br>

In [ ]:
# Display maps from pandas dataframes.
# Can zoom, pan, tilt

fig_height = 600

#################################### Map of All Patients ####################################

# Load lookup table with distinct building locations of organisations.
org_lookup = pd.read_csv('org_lookup.csv')
building_locs = org_lookup[['NAME_GENERALISED', 'LAT', 'LON']]

# Load patient locations.
patients = pd.read_csv('patients.csv')
patient_locs = patients[['Id', 'LAT', 'LON']]

print('All Patients')
fig = px.scatter_mapbox(patient_locs, lat="LAT", lon="LON", hover_name="Id",
                        color_discrete_sequence=["black"], zoom=6, height=fig_height)

fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

#################################### Same Map with 5km Threshold ####################################

# Patients Whose Closest Medical Organisation is Greater than 5 km.
patient_locs_gt_5_mask = dist_stats_and_closest['Percentile_0'] > 5
patient_locs_gt_5 = (pd.merge(dist_stats_and_closest[patient_locs_gt_5_mask],
                              all_combs_in,
                              how='left',
                              on='Patient ID')
                     [['Patient ID', 'Patient LAT', 'Patient LON']]
                    ).rename(columns={'Patient LAT' : 'LAT',
                                      'Patient LON' : 'LON'
                                     })
print(f'Patients Whose Closest Medical Organisation is Greater than 5km.')
fig = px.scatter_mapbox(patient_locs_gt_5, lat="LAT", lon="LON", hover_name="Patient ID",
                        color_discrete_sequence=["darkgreen"], zoom=6, height=fig_height)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

#################################### Same Map with 10km Threshold ####################################

# Patients Whose Closest Medical Organisation is Greater than 10 km.
patient_locs_gt_10_mask = dist_stats_and_closest['Percentile_0'] > 10
patient_locs_gt_10 = (pd.merge(dist_stats_and_closest[patient_locs_gt_10_mask],
                              all_combs_in,
                              how='left',
                              on='Patient ID')
                     [['Patient ID', 'Patient LAT', 'Patient LON']]
                    ).rename(columns={'Patient LAT' : 'LAT',
                                      'Patient LON' : 'LON'
                                     })
print(f'Patients Whose Closest Medical Organisation is Greater than 10km.')
fig = px.scatter_mapbox(patient_locs_gt_10, lat="LAT", lon="LON", hover_name="Patient ID",
                        color_discrete_sequence=["fuchsia"], zoom=6, height=fig_height)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

#################################### Map of a Few Clusters ####################################

# Patients whose closest medical organisation has the 9 highest number of organisation names for the same location.
# 'Building' is used to lump these organisations into one entity, noting that it may not be
# the best description.
org_lookup_top_9 = (pd.read_csv('org_lookup.csv')
                    .sort_values('COUNT', ascending=False)
                    .head(9)
                    .reset_index()
                    .rename(columns={'index' : 'Building Num',
                                     'NAME_GENERALISED' : 'Building ID',
                                     'LAT' : 'Building LAT',
                                     'LON' : 'Building LON'
                                    })
                   )

# Building Num is used to set the cluster colours.
# Colour is correlated to number of organisation 
org_lookup_top_9['Building Num'] = org_lookup_top_9['Building Num'] + 1

# Join with all patient location data to keep only patients whose closest organisation is in the top 9.
top_9_org_patients = (org_lookup_top_9.merge(dist_stats_and_closest,
                                            how='left',
                                            left_on='Building ID',
                                            right_on='Closest Med. Org.')
                      [['Building Num', 'Building ID', 'Patient ID']]
                     )

# Join previous result with patient locations.
top_9_org_patients = (top_9_org_patients.merge(patient_locs,
                                              how='left',
                                              left_on='Patient ID',
                                              right_on='Id')
                      .rename(columns={'LAT' : 'Patient LAT',
                                       'LON' : 'Patient LON'
                                      })
                      [['Building Num', 'Building ID', 'Patient ID', 'Patient LAT', 'Patient LON']]
                     )

# Plot maps with clusters of patients whose closest organisation is in the top 9.
print(f'Patients whose closest medical organisations have the highest number of organisation names per lat/long.')
print(f'Colour markers:\tpatients')
print(f'White markers:\torganisations')
zoom_dark = 7.5
fig = px.scatter_mapbox(top_9_org_patients,
                        lat='Patient LAT',
                        lon='Patient LON',
                        hover_name='Patient ID',
                        hover_data=['Building ID'],
                        color='Building Num',
                        zoom=zoom_dark,
                        height=fig_height)

# Overlay building locations with white markers.
fig2 = px.scatter_mapbox(org_lookup_top_9,
                         lat='Building LAT',
                         lon='Building LON',
                         hover_name='Building ID',
                         color_discrete_sequence=["white"],
                         zoom=zoom_dark,
                         height=fig_height)

fig.add_trace(fig2.data[0])

# Free access token needed for 'dark' map. See:
# https://plotly.com/python/mapbox-layers/#mapbox-access-tokens-and-when-you-need-them
    
# This one was found in example code at:
# https://chart-studio.plotly.com/~jb23626436/67/two-distinct-traces-on-scatter-mapbox/#/code
fig.update_layout(mapbox_style='dark',
                  mapbox_accesstoken='pk.eyJ1IjoiamIyMzYyNjQzNiIsImEiOiJjanBzamdxa3MxY2s4M3hvM2g2aTZhNDNsIn0.Ngy5pjDN9RllbdpwvQmvow')
fig.update_layout(margin={'r':0,'t':0,'l':0,'b':0})


<a id='boxplots_gt10km'></a>

# Box Plots for a Selected Distance Range
[Back to TOC](#toc)<br>

In [ ]:
# Patients Whose Closest Medical Organisation is Greater than 10 km.
patient_locs_gt10 = (pd.merge(dist_stats_and_closest[patient_locs_gt_10_mask],
                              all_combs_in,
                              how='left',
                              on='Patient ID')
                     .rename(columns={'Building ID' : 'Med. Org.'})
                     [['Patient ID', 'Med. Org.', 'Distance [km]', 'Closest Med. Org.']])

# Truncate the Patient IDs for visual clarity in the plot labels.
patient_locs_gt10['Patient ID'] = patient_locs_gt10['Patient ID'].str[:8]+'...'
display(patient_locs_gt10)

# Show distance distributions for patients whose closest medical organisation is > 10km away.
sns.set(font_scale = 1.5)
catplt = sns.catplot(x='Patient ID', y='Distance [km]', kind='box', col='Closest Med. Org.',
                     col_wrap=4, sharey=False, sharex=False, data=patient_locs_gt10)

# Increase gap between plot rows to prevent xlabels overlapping titles.
plt.subplots_adjust(hspace=0.5, wspace=0.4)

# Rotate xtick labels to avoid overlaps of each Patient ID label.
for axes in catplt.axes.flat:
    axes.set_xticklabels(axes.get_xticklabels(), rotation=45)
        
plt.show()


<a id='eda_modelling_auto'></a>

# EDA for Modelling - Automated Profile Reports
[Back to TOC](#toc)<br>

In [ ]:
# Generate EDA profile reports on all data.
profile_allergies = allergies.profile_report(title='Pandas Profiling Report - allergies')
profile_careplans = careplans.profile_report(title='Pandas Profiling Report - careplans')
profile_conditions = conditions.profile_report(title='Pandas Profiling Report - conditions')
profile_devices = devices.profile_report(title='Pandas Profiling Report - devices')
profile_encounters = encounters.profile_report(title='Pandas Profiling Report - encounters')
profile_imaging_studies = imaging_studies.profile_report(title='Pandas Profiling Report - imaging_studies')
profile_immunizations = immunizations.profile_report(title='Pandas Profiling Report - immunizations')
profile_medications = medications.profile_report(title='Pandas Profiling Report - medications')
profile_observations = observations.profile_report(title='Pandas Profiling Report - observations')
profile_organizations = organizations.profile_report(title='Pandas Profiling Report - organizations')
profile_patients = patients.profile_report(title='Pandas Profiling Report - patients')
profile_payers = payers.profile_report(title='Pandas Profiling Report - payers')
profile_payer_transitions = payer_transitions.profile_report(title='Pandas Profiling Report - payer_transitions')
# profile_procedures = procedures.profile_report(title='Pandas Profiling Report - procedures')
profile_providers = providers.profile_report(title='Pandas Profiling Report - providers')
profile_supplies = supplies.profile_report(title='Pandas Profiling Report - supplies')

# Save reports to file.
profile_allergies.to_file(output_file='allergies_pandas_profiling.html')
profile_careplans.to_file(output_file='careplans_pandas_profiling.html')
profile_conditions.to_file(output_file='conditions_pandas_profiling.html')
profile_devices.to_file(output_file='devices_pandas_profiling.html')
profile_encounters.to_file(output_file='encounters_pandas_profiling.html')
profile_imaging_studies.to_file(output_file='imaging_studies_pandas_profiling.html')
profile_immunizations.to_file(output_file='immunizations_pandas_profiling.html')
profile_medications.to_file(output_file='medications_pandas_profiling.html')
profile_observations.to_file(output_file='observations_pandas_profiling.html')
profile_organizations.to_file(output_file='organizations_pandas_profiling.html')
profile_patients.to_file(output_file='patients_pandas_profiling.html')
profile_payers.to_file(output_file='payers_pandas_profiling.html')
profile_payer_transitions.to_file(output_file='payer_transitions_pandas_profiling.html')
# profile_procedures.to_file(output_file='procedures_pandas_profiling.html')
profile_providers.to_file(output_file='providers_pandas_profiling.html')
profile_supplies.to_file(output_file='supplies_pandas_profiling.html')

<a id='eda_modelling_simple'></a>

# EDA for Modelling - Simple Pandas Dataframes
[Back to TOC](#toc)<br>

In [ ]:
# The Procedures data was unable to be processed by 'profile_report'.
# Checking basic things manually-ish.

def get_file_summary_data(label, data):
    print(f'\n\n############################################# {label.upper()} ######################################################')

    print(f'Number of Observations:\t{len(data)}')
    print(f'Number of Variables:\t{len(data.columns)}')
    
    # Get number and % of missing data.
    total = data.isnull().sum().sort_values(ascending=False)
    percent = (data.isnull().sum()/data.isnull().count()).sort_values(ascending=False)
    missing_data = pd.concat([total, percent], axis=1, keys=['Num Missing', '% Missing'])
    missing_data['% Missing'] = round(missing_data['% Missing'].apply(lambda x: x * 100), 2)
    missing_data.loc[missing_data['% Missing'] > 10][:10]
    
    # Duplicate rows.
    row_dupes = data.duplicated(keep='first').sum()
    row_dupes_pc = round(100.0*row_dupes/len(data), 3)
    print(f'\nDuplicate Rows:\t\t{row_dupes}')
    print(f'Duplicate Rows (%):\t{row_dupes_pc}%')

    # Distinct rows.
    distinct_rows = len(data.drop_duplicates())
    distinct_rows_pc = round(100.0*distinct_rows/len(data), 3)
    print(f'\nDistinct Rows:\t\t{distinct_rows}')
    print(f'Distinct Rows (%):\t{distinct_rows_pc}%')

    data_cols = data.columns
    data_cols_distinct = [len(data[col].unique()) for col in data_cols]

    data_distinct = pd.DataFrame(index=data_cols, data=data_cols_distinct, columns=['Num Distinct'])
    data_distinct['% Distinct'] = round(100*data_distinct['Num Distinct']/len(data), 2)

    summary_data = missing_data.merge(data_distinct, left_index=True, right_index=True)

    # Get stats on numerical columns.
    if not data.describe().empty:
        summary_data = summary_data.merge(data.describe().T,
                                          how='left',
                                          left_index=True,
                                          right_index=True)
    display(summary_data)

    
# All data files.
data_files = ['allergies','careplans','conditions','devices','encounters','imaging_studies','immunizations',
              'medications','observations','organizations','patients','payers','payer_transitions','procedures',
              'providers','supplies']

# Get summary data for all files.
for file in data_files[:]:
    # Load data file.
    file_name = file + '.csv'
    data = pd.read_csv(file_name)
    
    # Get data.
    get_file_summary_data(file, data)


<a id='pri_keys'></a>

# Primary Keys
[Back to TOC](#toc)<br>

In [ ]:
# Search for suitable candidate keys.
allergies_pri_key = ['PATIENT', 'CODE']
allergies_no_dupes = allergies.drop_duplicates(subset=allergies_pri_key)

careplans_pri_key = ['Id']
careplans_no_dupes = careplans.drop_duplicates(subset=careplans_pri_key)

conditions_pri_key = ['PATIENT', 'ENCOUNTER', 'CODE']
conditions_no_dupes = conditions.drop_duplicates(subset=conditions_pri_key)

devices_pri_key = ['PATIENT', 'CODE', 'START']
devices_no_dupes = devices.drop_duplicates(subset=devices_pri_key)

encounters_pri_key = ['Id']
encounters_no_dupes = encounters.drop_duplicates(subset=encounters_pri_key)

imaging_studies_pri_key = ['Id']
imaging_studies_no_dupes = imaging_studies.drop_duplicates(subset=imaging_studies_pri_key)

immunizations_pri_key = ['PATIENT', 'ENCOUNTER', 'CODE']
immunizations_no_dupes = immunizations.drop_duplicates(subset=immunizations_pri_key)

# Need to drop 3 duplicate rows first.
medications = medications.drop_duplicates()
medications_pri_key = ['ENCOUNTER', 'CODE', 'DISPENSES', 'BASE_COST']
medications_no_dupes = medications.drop_duplicates(subset=medications_pri_key)

# Need to drop 854 duplicate rows first.
observations = observations.drop_duplicates()
observations_pri_key = ['DATE', 'PATIENT', 'ENCOUNTER', 'CODE', 'VALUE']
observations_no_dupes = observations.drop_duplicates(subset=observations_pri_key)

organizations_pri_key = ['Id']
organizations_no_dupes = organizations.drop_duplicates(subset=organizations_pri_key)

patients_pri_key = ['Id']
patients_no_dupes = patients.drop_duplicates(subset=patients_pri_key)

payer_transitions_pri_key = ['PATIENT', 'START_YEAR']
payer_transitions_no_dupes = payer_transitions.drop_duplicates(subset=payer_transitions_pri_key)

payers_pri_key = ['Id']
payers_no_dupes = payers.drop_duplicates(subset=payers_pri_key)

procedures_pri_key = ['DATE', 'ENCOUNTER', 'CODE']
procedures_no_dupes = procedures.drop_duplicates(subset=procedures_pri_key)

providers_pri_key = ['Id']
providers_no_dupes = providers.drop_duplicates(subset=providers_pri_key)

supplies_pri_key = ['DATE', 'ENCOUNTER', 'CODE', 'QUANTITY']
supplies_no_dupes = supplies.drop_duplicates(subset=supplies_pri_key)

print(f'{len(allergies) - len(allergies_no_dupes):<5}duplicates with ALLERGIES candidate key:\t\tPrimary Key: {allergies_pri_key}')
print(f'{len(careplans) - len(careplans_no_dupes):<5}duplicates with CAREPLANS candidate key:\t\tPrimary Key: {careplans_pri_key}')
print(f'{len(conditions) - len(conditions_no_dupes):<5}duplicates with CONDITIONS candidate key:\t\tPrimary Key: {conditions_pri_key}')
print(f'{len(devices) - len(devices_no_dupes):<5}duplicates with DEVICES candidate key:\t\tPrimary Key: {devices_pri_key}')
print(f'{len(encounters) - len(encounters_no_dupes):<5}duplicates with ENCOUNTERS candidate key:\t\tPrimary Key: {encounters_pri_key}')
print(f'{len(imaging_studies) - len(imaging_studies_no_dupes):<5}duplicates with IMAGING_STUDIES candidate key:\tPrimary Key: {imaging_studies_pri_key}')
print(f'{len(immunizations) - len(immunizations_no_dupes):<5}duplicates with IMMUNIZATIONS candidate key:\tPrimary Key: {immunizations_pri_key}')
print(f'{len(medications) - len(medications_no_dupes):<5}duplicates with MEDICATIONS candidate key:\t\tPrimary Key: {medications_pri_key}')
print(f'{len(observations) - len(observations_no_dupes):<5}duplicates with OBSERVATIONS candidate key:\tPrimary Key: {observations_pri_key}')
print(f'{len(organizations) - len(organizations_no_dupes):<5}duplicates with ORGANIZATIONS candidate key:\tPrimary Key: {organizations_pri_key}')
print(f'{len(patients) - len(patients_no_dupes):<5}duplicates with PATIENTS key:\t\t\tPrimary Key: {patients_pri_key}')
print(f'{len(payer_transitions) - len(payer_transitions_no_dupes):<5}duplicates with PAYER_TRANSITIONS candidate key:\tPrimary Key: {payer_transitions_pri_key}')
print(f'{len(payers) - len(payers_no_dupes):<5}duplicates with PAYERS candidate key:\t\tPrimary Key: {payers_pri_key}')
print(f'{len(procedures) - len(procedures_no_dupes):<5}duplicates with PROCEDURES candidate key:\t\tPrimary Key: {procedures_pri_key}')
print(f'{len(providers) - len(providers_no_dupes):<5}duplicates with PROVIDERS candidate key:\t\tPrimary Key: {providers_pri_key}')
print(f'{len(supplies) - len(supplies_no_dupes):<5}duplicates with SUPPLIES candidate key:\t\tPrimary Key: {supplies_pri_key}')


<a id='create_ML_tables'></a>

# Create ML Tables
[Back to TOC](#toc)<br>

In [ ]:
pd.set_option('display.max_columns', None)

# Potentially interesting fields:
# allergies
# - code (1:1 with description)
# - start, stop (durations)
# - PK: ['PATIENT', 'CODE']
# - FK: ['PATIENT', 'ENCOUNTER']
allergies_ML = allergies[allergies_pri_key + ['START', 'STOP', 'ENCOUNTER', 'DESCRIPTION']]
display(get_file_summary_data('allergies_ML', allergies_ML))

# careplans
# - code (1:1 with description)
# - start, stop (durations)
# - PK: ['Id']
# - FK: ['PATIENT', 'ENCOUNTER']
careplans_ML = careplans[careplans_pri_key + ['START', 'STOP', 'CODE', 'PATIENT', 'ENCOUNTER', 'DESCRIPTION']]
display(get_file_summary_data('careplans_ML', careplans_ML))

# conditions
# - code (1:1 with description)
# - start, stop (durations)
# - PK: ['PATIENT', 'ENCOUNTER', 'CODE']
# - FK: ['PATIENT', 'ENCOUNTER']
conditions_ML = conditions[conditions_pri_key + ['START', 'STOP', 'DESCRIPTION']]
display(get_file_summary_data('conditions_ML', conditions_ML))

# devices
# - code (1:1 with description)
# - start, stop (durations)
# - PK: ['PATIENT', 'CODE', 'START']
# - FK: ['PATIENT', 'ENCOUNTER']
devices_ML = devices[devices_pri_key + ['STOP', 'ENCOUNTER', 'DESCRIPTION']]
display(get_file_summary_data('devices_ML', devices_ML))

# encounters
# - encounterclass
# - code (1:1 with description)
# - start, stop (durations)
# - PK: ['Id']
# - FK: ['PATIENT', 'ORGANIZATION', 'PROVIDER', 'PAYER']
encounters_ML = encounters[encounters_pri_key + ['START', 'PATIENT', 'ORGANIZATION', 'PROVIDER', 'PAYER', 'ENCOUNTERCLASS', 'DESCRIPTION']]
display(get_file_summary_data('encounters_ML', encounters_ML))

# imaging_studies
# - BODYSITE_CODE
# - MODALITY_CODE
# - PK: ['Id']
# - FK: ['PATIENT', 'ENCOUNTER']
imaging_studies_ML = imaging_studies[imaging_studies_pri_key + ['DATE', 'PATIENT', 'ENCOUNTER', 'BODYSITE_DESCRIPTION', 'MODALITY_DESCRIPTION']]
display(get_file_summary_data('imaging_studies_ML', imaging_studies_ML))

# immunizations
# - date
# - code
# - PK: ['PATIENT', 'ENCOUNTER', 'CODE']
# - FK: ['PATIENT', 'ENCOUNTER']
immunizations_ML = immunizations[immunizations_pri_key + ['DATE', 'DESCRIPTION']]
display(get_file_summary_data('immunizations_ML', immunizations_ML))

# medications
# - start, stop (durations)
# - code
# - BASE_COST
# - PAYER_COVERAGE
# - DISPENSES
# - TOTALCOST
# - PK: ['ENCOUNTER', 'CODE', 'DISPENSES', 'BASE_COST']
# - FK: ['PATIENT', 'PAYER', ENCOUNTER']
medications_ML = medications[medications_pri_key + ['START', 'STOP', 'PAYER_COVERAGE', 'TOTALCOST', 'PATIENT', 'DESCRIPTION']]
display(get_file_summary_data('medications_ML', medications_ML))

# observations
# - code
# - value
# - units
# - type
# - date
# - PK: ['DATE', 'PATIENT', 'ENCOUNTER', 'CODE', 'VALUE']
# - FK: ['PATIENT', 'ENCOUNTER']
observations_ML = observations[observations_pri_key + ['UNITS', 'TYPE', 'DESCRIPTION']]
display(get_file_summary_data('observations_ML', observations_ML))

# organizations
# - revenue
# - utilization
# - city
# - PK: ['Id']
# - FK: ['PATIENT', 'ENCOUNTER']
organizations_ML = organizations[organizations_pri_key + ['REVENUE', 'UTILIZATION', 'CITY']]
display(get_file_summary_data('organizations_ML', organizations_ML))

# patients
# - death date (although only ~2k non-null)
# - birth date (get age)
# - gender
# - ethnicity
# - race
# - healthcare expenses / coverage (seem very high)
# - county
# - marital
# - PK: ['Id']
# - FK: []
patients_ML = patients[patients_pri_key + ['BIRTHDATE', 'DEATHDATE', 'GENDER',
                                           'ETHNICITY', 'RACE', 'HEALTHCARE_EXPENSES',
                                           'HEALTHCARE_COVERAGE', 'COUNTY', 'MARITAL']]
display(get_file_summary_data('patients_ML', patients_ML))

# payer_transitions
# - start_year
# - end_year
# - PK: ['PATIENT', 'START_YEAR']
# - FK: ['PATIENT', 'PAYER']
payer_transitions_ML = payer_transitions[payer_transitions_pri_key + ['END_YEAR', 'PAYER', 'OWNERSHIP']]
display(get_file_summary_data('payer_transitions_ML', payer_transitions_ML))

# payers
# - AMOUNT_COVERED
# - AMOUNT_UNCOVERED
# - REVENUE
# - COVERED_ENCOUNTERS
# - UNCOVERED_ENCOUNTERS
# - COVERED_MEDICATIONS
# - UNCOVERED_MEDICATIONS
# - COVERED_PROCEDURES
# - UNCOVERED_PROCEDURES
# - COVERED_IMMUNIZATIONS
# - UNCOVERED_IMMUNIZATIONS
# - UNIQUE_CUSTOMERS
# - QOLS_AVG
# - MEMBER_MONTHS
# - PK: ['Id']
# - FK: []
payers_ML = payers[payers_pri_key + ['AMOUNT_COVERED','AMOUNT_UNCOVERED','REVENUE',
                                     'COVERED_ENCOUNTERS','UNCOVERED_ENCOUNTERS','COVERED_MEDICATIONS',
                                     'UNCOVERED_MEDICATIONS','COVERED_PROCEDURES','UNCOVERED_PROCEDURES',
                                     'COVERED_IMMUNIZATIONS','UNCOVERED_IMMUNIZATIONS','UNIQUE_CUSTOMERS',
                                     'QOLS_AVG','MEMBER_MONTHS']]
display(get_file_summary_data('payers_ML', payers_ML))

# procedures
# - date
# - code
# - BASE_COST
# - PK: ['DATE', 'ENCOUNTER', 'CODE']
# - FK: ['PATIENT', 'ENCOUNTER']
procedures_ML = procedures[procedures_pri_key + ['PATIENT', 'BASE_COST', 'DESCRIPTION']]
display(get_file_summary_data('procedures_ML', procedures_ML))

# providers
# - speciality
# - city
# - utilization
# - PK: ['Id']
# - FK: ['ORGANIZATION']
providers_ML = providers[providers_pri_key + ['SPECIALITY', 'CITY', 'UTILIZATION', 'ORGANIZATION']]
# display(get_file_summary_data('providers_ML', providers_ML))

# supplies
# - code
# - date
# - Note: high cross-correlation between QUANTITY and CODE.
# - PK: ['DATE', 'ENCOUNTER', 'CODE', 'QUANTITY']
# - FK: ['PATIENT', 'ENCOUNTER']
supplies_ML = supplies[supplies_pri_key + ['PATIENT', 'DESCRIPTION']]
# display(get_file_summary_data('supplies_ML', supplies_ML))

# New data with calculated patient-org distances.
# pat_bldg_dist_stats
# - PK: ['Patient ID']
# - FKs: ['Closest Building ID'] (same as Building ID)
pat_bldg_dist_stats = pd.read_csv('pat_bldg_dist_stats.csv')

# org_buildings
# - PK: ['Building ID']
# - FKs: ['Building LAT', 'Building LON']
org_buildings = pd.read_csv('org_lookup.csv')

pat_bldg_dist_stats_ML = (pat_bldg_dist_stats
                          .rename(columns={'Closest Med. Org.' : 'Closest Building ID',
                                           'Percentile_50' : 'Median Distance',
                                          })
                         )[['Patient ID', 'Closest Building ID', 'Median Distance']]

org_buildings_ML = (org_buildings
                    .rename(columns={'NAME_GENERALISED' : 'Building ID',
                                     'LAT' : 'Building LAT',
                                     'LON' : 'Building LON',
                                    })
                   )[['Building ID', 'Building LAT', 'Building LON']]


display(get_file_summary_data('pat_bldg_dist_stats_ML', pat_bldg_dist_stats_ML))
display(get_file_summary_data('org_buildings_ML', org_buildings_ML))


<a id='pivot_patient'></a>

# Create and Join Pivot Tables for All Tables on PATIENT
[Back to TOC](#toc)<br>

In [ ]:
pd.set_option('display.min_rows', 10)

# Create pivot tables to count

print('\n############################## ALLERGIES ##############################')
allergies_pivot = pd.pivot_table(allergies_ML, values='START', index='PATIENT',
                                 columns='DESCRIPTION', aggfunc='count', fill_value=0)
allergies_pivot = allergies_pivot.rename_axis(None, axis=1)
display(allergies_pivot)
# display(careplans_pivot.sum())

print('\n############################## CAREPLANS ##############################')
careplans_pivot = pd.pivot_table(careplans_ML, values='START', index='PATIENT',
                                 columns='DESCRIPTION', aggfunc='count', fill_value=0)
careplans_pivot = careplans_pivot.rename_axis(None, axis=1)
display(careplans_pivot)
# display(careplans_pivot.sum())

print('\n############################## CONDITIONS ##############################')
conditions_pivot = pd.pivot_table(conditions_ML, values='START', index='PATIENT',
                                 columns='DESCRIPTION', aggfunc='count', fill_value=0)
conditions_pivot = conditions_pivot.rename_axis(None, axis=1)
display(conditions_pivot)
# display(conditions_pivot.sum())

print('\n############################## DEVICES ##############################')
devices_pivot = pd.pivot_table(devices_ML, values='START', index='PATIENT',
                                 columns='DESCRIPTION', aggfunc='count', fill_value=0)
devices_pivot = devices_pivot.rename_axis(None, axis=1)
display(devices_pivot)
# display(devices_pivot.sum())

print('\n############################## ENCOUNTERS ##############################')
encounters_pivot = pd.pivot_table(encounters_ML, values='START', index='PATIENT',
                                 columns='DESCRIPTION', aggfunc='count', fill_value=0)
encounters_pivot = encounters_pivot.rename_axis(None, axis=1)
display(encounters_pivot)
# display(encounters_pivot.sum())

print('\n############################## IMAGING_STUDIES ##############################')
imaging_studies_pivot = pd.pivot_table(imaging_studies_ML, values='DATE', index='PATIENT',
                                 columns='BODYSITE_DESCRIPTION', aggfunc='count', fill_value=0)
imaging_studies_pivot = imaging_studies_pivot.rename_axis(None, axis=1)
display(imaging_studies_pivot)
# display(imaging_studies_pivot.sum())

print('\n############################## IMMUNIZATIONS ##############################')
immunizations_pivot = pd.pivot_table(immunizations_ML, values='DATE', index='PATIENT',
                                 columns='DESCRIPTION', aggfunc='count', fill_value=0)
immunizations_pivot = immunizations_pivot.rename_axis(None, axis=1)
display(immunizations_pivot)
# display(immunizations_pivot.sum())

print('\n############################## MEDICATIONS ##############################')
medications_pivot = pd.pivot_table(medications_ML, values='START', index='PATIENT',
                                 columns='DESCRIPTION', aggfunc='count', fill_value=0)
medications_pivot = medications_pivot.rename_axis(None, axis=1)
display(medications_pivot)
# display(medications_pivot.sum())

print('\n############################## OBSERVATIONS ##############################')
# display(observations['DESCRIPTION'].value_counts())
observations_pivot = pd.pivot_table(observations_ML, values='DATE', index='PATIENT',
                                 columns='DESCRIPTION', aggfunc='count', fill_value=0)
observations_pivot = observations_pivot.rename_axis(None, axis=1)
display(observations_pivot)
# display(observations_pivot.sum())

# # NO PATIENT LEVEL DATA. NOT RELEVANT TO PIVOT.
# display(organizations['DESCRIPTION'].value_counts())

print('\n############################## PATIENTS_GENDER ##############################')
# display(patients['GENDER'].value_counts())
patients_gender_pivot = pd.pivot_table(patients_ML, values='BIRTHDATE', index='Id',
                                 columns='GENDER', aggfunc='count', fill_value=0)
patients_gender_pivot = patients_gender_pivot.rename_axis(None, axis=1)
display(patients_gender_pivot)
# display(patients_gender_pivot.sum())

print('\n############################## PATIENTS_ETHNICITY ##############################')
# display(patients['ETHNICITY'].value_counts())
patients_ethn_pivot = pd.pivot_table(patients_ML, values='BIRTHDATE', index='Id',
                                 columns='ETHNICITY', aggfunc='count', fill_value=0)
patients_ethn_pivot = patients_ethn_pivot.rename_axis(None, axis=1)
display(patients_ethn_pivot)
# display(patients_ethn_pivot.sum())

print('\n############################## PATIENTS_RACE ##############################')
# display(patients['RACE'].value_counts())
patients_race_pivot = pd.pivot_table(patients_ML, values='BIRTHDATE', index='Id',
                                 columns='RACE', aggfunc='count', fill_value=0)
patients_race_pivot = patients_race_pivot.rename_axis(None, axis=1)
display(patients_race_pivot)
# display(patients_race_pivot.sum())

print('\n############################## PATIENTS_COUNTY ##############################')
# display(patients['COUNTY'].value_counts())
patients_county_pivot = pd.pivot_table(patients_ML, values='BIRTHDATE', index='Id',
                                 columns='COUNTY', aggfunc='count', fill_value=0)
patients_county_pivot = patients_county_pivot.rename_axis(None, axis=1)
display(patients_county_pivot)
# display(patients_county_pivot.sum())

print('\n############################## PATIENTS_MARITAL ##############################')
# display(patients['MARITAL'].value_counts())
patients_marital_pivot = pd.pivot_table(patients_ML, values='BIRTHDATE', index='Id',
                                 columns='MARITAL', aggfunc='count', fill_value=0)
patients_marital_pivot = patients_marital_pivot.rename_axis(None, axis=1)
display(patients_marital_pivot)
# display(patients_marital_pivot.sum())

# # NO PATIENT LEVEL DATA. NOT RELEVANT TO PIVOT.
# display(payers['DESCRIPTION'].value_counts())

print('\n############################## PAYER_TRANSITIONS ##############################')
payer_transitions_pivot = pd.pivot_table(payer_transitions_ML, values='START_YEAR', index='PATIENT',
                                 columns='OWNERSHIP', aggfunc='count', fill_value=0)
payer_transitions_pivot = payer_transitions_pivot.rename_axis(None, axis=1)
display(payer_transitions_pivot)
# display(payer_transitions_pivot.sum())

print('\n############################## PROCEDURES ##############################')
procedures_pivot = pd.pivot_table(procedures_ML, values='DATE', index='PATIENT',
                                 columns='DESCRIPTION', aggfunc='count', fill_value=0)
procedures_pivot = procedures_pivot.rename_axis(None, axis=1)
display(procedures_pivot)
# display(procedures_pivot.sum())

# # NO PATIENT LEVEL DATA. NOT RELEVANT TO PIVOT.
# display(providers['DESCRIPTION'].value_counts())

print('\n############################## SUPPLIES ##############################')
supplies_pivot = pd.pivot_table(supplies_ML, values='DATE', index='PATIENT',
                                 columns='DESCRIPTION', aggfunc='count', fill_value=0)
supplies_pivot = supplies_pivot.rename_axis(None, axis=1)
display(supplies_pivot)
# display(supplies_pivot.sum())


# Join all patient pivot tables.
all_pivots_ML = []
all_pivots_ML = (patients_ML[['Id', 'BIRTHDATE', 'DEATHDATE', 'HEALTHCARE_EXPENSES', 'HEALTHCARE_COVERAGE']]
                 .copy(deep=True)
                 .rename(columns={'Id' : 'PATIENT'})
                )

# Convert dates to 'days since' fields and remove absolute dates.
ref_date = pd.Timestamp(year=2021, month=5, day=20, hour=0)
all_pivots_ML.BIRTHDATE = pd.to_datetime(all_pivots_ML.BIRTHDATE)
all_pivots_ML.DEATHDATE = pd.to_datetime(all_pivots_ML.DEATHDATE)
all_pivots_ML['DAYS_SINCE_BIRTH'] = (ref_date - all_pivots_ML.BIRTHDATE) / np.timedelta64(1, 'D')

# all_pivots_ML['DAYS_SINCE_DEATH'] = ref_date - all_pivots_ML.DEATHDATE
type_days_since_birth = type(all_pivots_ML['DAYS_SINCE_BIRTH'][0])
# print(f'DAYS_SINCE_BIRTH type: {type_days_since_birth}')

patient_cols = list(all_pivots_ML.columns)
patient_cols.remove('BIRTHDATE')
patient_cols.remove('DEATHDATE')
all_pivots_ML = all_pivots_ML[patient_cols]

patients_gender_pivot = patients_gender_pivot.reset_index().rename(columns={'Id' : 'PATIENT'})
patients_ethn_pivot = patients_ethn_pivot.reset_index().rename(columns={'Id' : 'PATIENT'})
patients_race_pivot = patients_race_pivot.reset_index().rename(columns={'Id' : 'PATIENT'})
patients_county_pivot = patients_county_pivot.reset_index().rename(columns={'Id' : 'PATIENT'})
patients_marital_pivot = patients_marital_pivot.reset_index().rename(columns={'Id' : 'PATIENT'})

all_pivots_ML = all_pivots_ML.merge(patients_gender_pivot, how='left', on='PATIENT')
all_pivots_ML = all_pivots_ML.merge(patients_ethn_pivot, how='left', on='PATIENT')
all_pivots_ML = all_pivots_ML.merge(patients_race_pivot, how='left', on='PATIENT')
all_pivots_ML = all_pivots_ML.merge(patients_county_pivot, how='left', on='PATIENT')
all_pivots_ML = all_pivots_ML.merge(patients_marital_pivot, how='left', on='PATIENT')
all_pivots_ML = all_pivots_ML.merge(allergies_pivot.reset_index(), how='left', on='PATIENT')
all_pivots_ML = all_pivots_ML.merge(careplans_pivot.reset_index(), how='left', on='PATIENT')
all_pivots_ML = all_pivots_ML.merge(conditions_pivot.reset_index(), how='left', on='PATIENT')
all_pivots_ML = all_pivots_ML.merge(devices_pivot.reset_index(), how='left', on='PATIENT')
all_pivots_ML = all_pivots_ML.merge(encounters_pivot.reset_index(), how='left', on='PATIENT')
all_pivots_ML = all_pivots_ML.merge(imaging_studies_pivot.reset_index(), how='left', on='PATIENT')
all_pivots_ML = all_pivots_ML.merge(immunizations_pivot.reset_index(), how='left', on='PATIENT')
all_pivots_ML = all_pivots_ML.merge(medications_pivot.reset_index(), how='left', on='PATIENT')
all_pivots_ML = all_pivots_ML.merge(observations_pivot.reset_index(), how='left', on='PATIENT')
all_pivots_ML = all_pivots_ML.merge(payer_transitions_pivot.reset_index(), how='left', on='PATIENT')
all_pivots_ML = all_pivots_ML.merge(procedures_pivot.reset_index(), how='left', on='PATIENT')
all_pivots_ML = all_pivots_ML.merge(supplies_pivot.reset_index(), how='left', on='PATIENT')

all_pivots_ML = all_pivots_ML.rename(columns={'M_x' : 'Male',
                                              'F' : 'Female',
                                              'M_y' : 'Married',
                                              'S' : 'Single'})

display(all_pivots_ML)

<a id='rfc_prep'></a>

# Random Forest Classifier Data Preparation
[Back to TOC](#toc)<br>

In [ ]:
# Fill NaN values in merged pivot data with zero.
# Leave NaN in other columns alone.
patient_cols = ['PATIENT', 'DAYS_SINCE_BIRTH', 'HEALTHCARE_EXPENSES', 'HEALTHCARE_COVERAGE']
for col in all_pivots_ML.columns:
    if col not in patient_cols:
        all_pivots_ML[col] = all_pivots_ML[col].fillna(0)
        
# Cut out HEALTHCARE_COVERAGE outliers.
HEALTHCARE_COVERAGE_outlier_thresh = np.percentile(all_pivots_ML['HEALTHCARE_COVERAGE'], 95)
HEALTHCARE_COVERAGE_outlier_mask = all_pivots_ML['HEALTHCARE_COVERAGE'] < HEALTHCARE_COVERAGE_outlier_thresh
all_pivots_ML = all_pivots_ML[HEALTHCARE_COVERAGE_outlier_mask]

# Remove zeros in HEALTHCARE_COVERAGE after preliminary visualisation of log-transformed distribution.
# Only 301 rows lost.
HEALTHCARE_COVERAGE_zero_mask = all_pivots_ML['HEALTHCARE_COVERAGE'] <= 0
all_pivots_ML = all_pivots_ML[~HEALTHCARE_COVERAGE_zero_mask]

# Log transformations of skewed data.
all_pivots_ML['HEALTHCARE_COVERAGE'] = np.log(all_pivots_ML['HEALTHCARE_COVERAGE'] + 0.001)

# Normalise float data.
# https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html
# Normalisation Formula:
# X_std = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))
# X_scaled = X_std * (max - min) + min

# Set up scaling object.
scaler_HEALTHCARE_EXPENSES = MinMaxScaler()
scaler_HEALTHCARE_COVERAGE = MinMaxScaler()
scaler_DAYS_SINCE_BIRTH = MinMaxScaler()

# Scale floating point data.
HEALTHCARE_EXPENSES_scaled = scaler_HEALTHCARE_EXPENSES.fit_transform(all_pivots_ML['HEALTHCARE_EXPENSES'].values.reshape(-1, 1))
HEALTHCARE_COVERAGE_scaled = scaler_HEALTHCARE_COVERAGE.fit_transform(all_pivots_ML['HEALTHCARE_COVERAGE'].values.reshape(-1, 1))
DAYS_SINCE_BIRTH_scaled = scaler_DAYS_SINCE_BIRTH.fit_transform(all_pivots_ML['DAYS_SINCE_BIRTH'].values.reshape(-1, 1))

all_pivots_ML['HEALTHCARE_EXPENSES'] = HEALTHCARE_EXPENSES_scaled
all_pivots_ML['HEALTHCARE_COVERAGE'] = HEALTHCARE_COVERAGE_scaled
all_pivots_ML['DAYS_SINCE_BIRTH'] = DAYS_SINCE_BIRTH_scaled

print(f'\nTransformations (all normalised):')
print(f'HEALTHCARE_EXPENSES:\tNot transformed.')
print(f'HEALTHCARE_COVERAGE:\tLog transformed.')
print(f'DAYS_SINCE_BIRTH:\tNot transformed.')

# Plot histograms for transformed and normalised floating point fields.
all_pivots_ML.hist(column=['HEALTHCARE_EXPENSES', 'HEALTHCARE_COVERAGE', 'DAYS_SINCE_BIRTH'],
                   bins=50, layout=(1,3), figsize=(20,5))

# Check for NaN in processed float variables.
HEALTHCARE_EXPENSES_nan_mask = all_pivots_ML['HEALTHCARE_EXPENSES'].isnull()
HEALTHCARE_COVERAGE_nan_mask = all_pivots_ML['HEALTHCARE_COVERAGE'].isnull()
DAYS_SINCE_BIRTH_nan_mask = all_pivots_ML['DAYS_SINCE_BIRTH'].isnull()

# Cross Correlation.
# Eliminate all fields that have a cross-correlation > 80%... indicates a high likelihood of confounding variables.
# Also, cuts down number of columns down from ~900 to ~400
# Note that this approach would be better if it kept one of the variables, however given the large column count,
# it is expected that not too much will be lost for the current scope of work.
all_pivots_ML_corr = all_pivots_ML.corr()
xcorr_80 = all_pivots_ML_corr > 0.8
xcorr_80_sum_index = xcorr_80.sum().index
xcorr_80_sum_values = xcorr_80.sum().values

# Create list of fields to keep.
lower_xcorr_cols = [xcorr_80_sum_index[i] for i in range(len(xcorr_80_sum_index)) if xcorr_80_sum_values[i] == 1]

# Remove undesired fields for the initial ML model.
all_pivots_ML = all_pivots_ML[lower_xcorr_cols]


# Shuffle rows and update the final table to be used as the preliminary ML model.
all_pivots_ML = all_pivots_ML.sample(frac=1).reset_index(drop=True)
initial_ML_data = all_pivots_ML.copy(deep=True)
display(initial_ML_data)


<a id='rfc_model'></a>

# Random Forest Classifier Modelling
[Back to TOC](#toc)<br>

In [ ]:
# Find all fields with 'finding'.
finding_col = [col for col in list(initial_ML_data.columns) if 'finding' in col.lower()]

# From previous plots of several fields with 'finding' in the name,
# 'Body mass index 30+ - obesity (finding)' has a good balance between levels.
initial_ML_data.hist(column='Body mass index 30+ - obesity (finding)', bins=3)

In [ ]:
# Extract labels.
labels = initial_ML_data['Body mass index 30+ - obesity (finding)']

# Extract features.
features = initial_ML_data.drop('Body mass index 30+ - obesity (finding)', axis=1)

# Split data into training / test subsets.
train_features, test_features, train_labels, test_labels = train_test_split(features,
                                                                            labels,
                                                                            test_size=0.3,
                                                                            random_state=42)

# Create random forest Classifier object.
clf = RandomForestClassifier(n_jobs=2, random_state=42)

# Train the Classifier.
clf.fit(train_features, train_labels)

# Apply the model to the test data.
preds = clf.predict(test_features)

# Predicted probabilities.
probs = clf.predict_proba(test_features)

# Confusion matrix
conf_matrix = pd.crosstab(test_labels, preds, rownames=['Actual'], colnames=['Predicted'])
display(conf_matrix)

# Adjust number of rows to display.
show_rows = 40
pd.set_option('display.max_rows', show_rows)
pd.set_option('display.min_rows', show_rows)

# Get features importance scores
feat_imp = (pd.DataFrame(zip(train_features, clf.feature_importances_), columns=['Feature', 'Importance'])
            .sort_values('Importance', ascending=False)
            .reset_index(drop=True))

# Precision, Recall, F1
print(classification_report(test_labels, preds, target_names=['NOT BMI 30+', 'BMI 30+']))


In [ ]:
# ROC and AUC
plt.rcParams.update({'font.size': 12})
auc = roc_auc_score(test_labels, probs[:,1])
fpr, tpr, thresholds = roc_curve(test_labels, probs[:,1])
plt.plot(fpr, tpr, label=f'BMI 30+    AUC: {round(auc, 2)}')
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='lower right')


In [ ]:
# Get most important features that comprise cumulatively 95% of all importance.
feat_imp['Cumulative Importance'] = feat_imp['Importance'].cumsum()
cumul_imp_mask = feat_imp['Cumulative Importance'] < 0.95
feat_imp = feat_imp[cumul_imp_mask]
display(feat_imp)

# Plot feature importances.
plt.rcParams.update({'font.size': 40})
feat_imp.plot.barh(y='Importance', x='Feature', figsize=(20,60))
plt.gca().invert_yaxis()

<a id='quasi_ERD'></a>

# Quasi ERD
[Back to TOC](#toc)<br>

In [ ]:
from IPython.core.display import SVG
SVG(filename='ERD.svg')
# Use scrolls to 

In [ ]:
# Capture notebook run time.
end_time = time()
print(f'Total elapsed time [s]:\t{round(end_time - start_time, 2)}')
